In [1]:
using Flux, LinearAlgebra

In [2]:
using CUDA

In [21]:
H_size = 16

16

In [22]:
Kp = rand(ComplexF32, H_size,H_size)
K = rand(ComplexF32, H_size,H_size)
Ht = rand(ComplexF32, H_size,H_size)
Kp = Hermitian(Kp)
K = Hermitian(K)
Ht = Hermitian(Ht)
Kp_g = gpu(Kp)
K_g = gpu(K)
Ht_g = gpu(Ht)

16×16 CuArray{ComplexF32, 2, CUDA.Mem.DeviceBuffer}:
  0.839159+0.0im        0.791178+0.92915im    …   0.255124+0.636849im
  0.791178-0.92915im    0.503697+0.0im            0.347082+0.413243im
  0.846683-0.334754im   0.948065-0.0756429im      0.502709+0.962857im
  0.747884-0.846405im   0.958799-0.324977im       0.747326+0.419077im
 0.0788435-0.314229im  0.0356427-0.562537im        0.43728+0.0178365im
  0.207245-0.239601im  0.0831298-0.123366im   …  0.0497276+0.268114im
  0.466356-0.814077im   0.798071-0.283358im      0.0945815+0.412606im
  0.703873-0.74163im   0.0299082-0.818892im      0.0435041+0.765374im
  0.101615-0.842481im   0.323173-0.992504im       0.115635+0.341516im
    0.9924-0.810692im   0.539046-0.287695im       0.376597+0.595861im
  0.151242-0.409853im  0.0501673-0.881003im   …  0.0459595+0.825191im
  0.143205-0.116088im   0.221634-0.256928im       0.116766+0.181859im
  0.147223-0.830604im   0.140517-0.346802im       0.728939+0.70857im
  0.513895-0.716265im   0.966294-0.89

In [23]:
@time HF = K'*(Ht - 1.0im*Kp)*K

  0.000039 seconds (8 allocations: 46.500 KiB)


16×16 Matrix{ComplexF64}:
  66.9338-71.5519im    83.7097-52.6268im  …  96.2548+58.9043im
  51.3561-94.1284im    76.0312-76.5818im     117.032+34.9033im
  32.3679-81.2007im    51.3918-69.1177im       94.79+16.9229im
  45.9763-103.649im    72.7873-86.8974im     125.269+28.2318im
  35.2342-85.0967im    58.2396-72.6811im     104.313+18.3672im
   31.367-92.1965im    52.8503-79.2519im  …   105.92+15.1476im
  16.1801-109.922im    47.3932-101.813im     122.749-3.69028im
  19.0322-107.311im    44.3719-102.183im     123.472-3.98623im
  4.99482-113.257im    33.7594-107.202im     121.084-21.7155im
 -18.2603-96.6056im    3.69716-98.2237im     100.859-46.9062im
 -21.1488-94.8318im  0.0491369-100.977im  …  103.905-52.8378im
  -18.185-86.0723im   -2.33142-92.9304im     92.4012-44.9927im
 -33.8809-82.7201im   -14.6325-96.8739im     87.9344-61.5956im
 -56.9904-90.0241im   -37.0684-107.152im     89.2277-89.6242im
 -79.0085-70.4634im    -61.017-90.1885im     60.5363-106.689im
  -59.273-93.9723im   -37.036

In [25]:
@time HF_g = K_g'*(Ht_g - 1.0im*Kp_g)*K_g

  0.000273 seconds (149 allocations: 7.547 KiB)


16×16 CuArray{ComplexF64, 2, CUDA.Mem.DeviceBuffer}:
  66.9338-71.5519im    83.7097-52.6268im  …  96.2548+58.9043im
  51.3561-94.1284im    76.0312-76.5818im     117.032+34.9033im
  32.3679-81.2007im    51.3918-69.1177im       94.79+16.9229im
  45.9763-103.649im    72.7873-86.8974im     125.269+28.2318im
  35.2342-85.0967im    58.2396-72.6811im     104.313+18.3672im
   31.367-92.1965im    52.8503-79.2519im  …   105.92+15.1476im
  16.1801-109.922im    47.3932-101.813im     122.749-3.69028im
  19.0322-107.311im    44.3719-102.183im     123.472-3.98623im
  4.99482-113.257im    33.7594-107.202im     121.084-21.7155im
 -18.2603-96.6056im    3.69716-98.2237im     100.859-46.9062im
 -21.1488-94.8318im  0.0491369-100.977im  …  103.905-52.8378im
  -18.185-86.0723im   -2.33142-92.9304im     92.4012-44.9927im
 -33.8809-82.7201im   -14.6325-96.8739im     87.9344-61.5956im
 -56.9904-90.0241im   -37.0684-107.152im     89.2277-89.6242im
 -79.0085-70.4634im    -61.017-90.1885im     60.5363-106.689im
  

In [13]:
n_size = 128

128

In [20]:
model = Chain(Dense(4, n_size, tanh), Dense(n_size, n_size, tanh), Dense(n_size, n_size, tanh), Dense(n_size, 6))

Chain(
  Dense(4 => 128, tanh),                # 640 parameters
  Dense(128 => 128, tanh),              # 16_512 parameters
  Dense(128 => 128, tanh),              # 16_512 parameters
  Dense(128 => 6),                      # 774 parameters
)                   # Total: 8 arrays, 34_438 parameters, 135.023 KiB.

In [21]:
model_g = fmap(cu, model)

Chain(
  Dense(4 => 128, tanh),                # 640 parameters
  Dense(128 => 128, tanh),              # 16_512 parameters
  Dense(128 => 128, tanh),              # 16_512 parameters
  Dense(128 => 6),                      # 774 parameters
)                   # Total: 8 arrays, 34_438 parameters, 1.109 KiB.

In [4]:
x = [1.0, 2.0, 0.5, 0.1]

4-element Vector{Float64}:
 1.0
 2.0
 0.5
 0.1

In [16]:
x = gpu(x)

4-element CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}:
 1.0
 2.0
 0.5
 0.1

In [22]:
@time model_g(x)

  0.006424 seconds (5.93 k allocations: 334.892 KiB, 96.28% compilation time)


6-element CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}:
 -0.10658946
 -0.25949526
 -0.384801
 -0.23403132
  0.12074848
  0.09428628

In [7]:
model = Chain(Dense(4, 64, tanh), Dense(64, 64, tanh), Dense(64, 6))

Chain(
  Dense(4 => 64, tanh),                 # 320 parameters
  Dense(64 => 64, tanh),                # 4_160 parameters
  Dense(64 => 6),                       # 390 parameters
)                   # Total: 6 arrays, 4_870 parameters, 19.398 KiB.

In [18]:
y = [1.0, 2.0, 0.5, 0.1]

4-element Vector{Float64}:
 1.0
 2.0
 0.5
 0.1

In [23]:
@time model(y)

  0.005960 seconds (5.80 k allocations: 334.134 KiB, 99.08% compilation time)


6-element Vector{Float64}:
 -0.10658943090832201
 -0.25949524817210207
 -0.38480098976911187
 -0.2340313191338875
  0.12074845135298967
  0.09428631872700924